In [6]:
import gnupg
rootpath = 'C:\\Users\\prashant\\Desktop\\PGP\\FTSol'
keypath = rootpath + '\gnupg'
mediapath = rootpath + '\\media'
S_Email = 'jones.thayil@gmail.com'
S_Passkey = 'Pass@123'
S_Type = 'RSA'
S_Length = '2048'
C_Email = 'jones.thayil@outlook.com'
C_Passkey = 'Pass@123'
C_Type = 'RSA'
C_Length = '2048'
gpg = gnupg.GPG(gnupghome=keypath)
gpg.encoding = 'utf-8'
def generatekey(ftype='Client'):
    if ftype == 'Server':
        input_data = gpg.gen_key_input(
            name_email=S_Email,
            passphrase=S_Passkey,
            key_type=S_Type,
            key_length=S_Length,
        )
    else:
        input_data = gpg.gen_key_input(
            name_email=C_Email,
            passphrase=C_Passkey,
            key_type=C_Type,
            key_length=C_Length,
        )
    newkey = gpg.gen_key(input_data)
    print(newkey)
    return str(newkey)

S_Key = generatekey('Server')
C_Key = generatekey()
print(type(S_Key))
print(type(C_Key))
# print(gpg.list_keys())
# print(gpg.list_keys(True))
def ftecrt(filename):
    print('am in')
    sigfile = mediapath + "\\signatures" + filename.split(".")[0] + ".sig"
    resultfile = mediapath + "\\encrypted" + filename.split(".")[0] + ".gpg"
    with open(mediapath + filename, "rb") as f:
        stream = gpg.sign_file(f,passphrase=S_Passkey,detach = True, output=sigfile)
    with open(mediapath + filename, 'rb') as f:
        status = gpg.encrypt_file(f,recipients=[C_Email],output= resultfile)
        print("ok: ", status.ok)
        print("status: ", status.status)
        print("stderr: ", status.stderr)
    print('Test On')
    a = gpg.encrypt('data', recipients=[C_Email], sign=S_Key, passphrase=S_Passkey)
    print('Test Off', a)
    return str(resultfile)
ftecrt('\\terms.txt')

41A0F38BFCCCD71A583B1019E47E06C9175C3412
A8F8B4BFA29016179D1DFA4ABB44DA091B258C40
<class 'str'>
<class 'str'>
am in
ok:  True
status:  encryption ok
stderr:  [GNUPG:] KEY_CONSIDERED 151475F219518CE554B2EBE8895B282049DB9BFC 0
[GNUPG:] KEY_CONSIDERED 151475F219518CE554B2EBE8895B282049DB9BFC 2
[GNUPG:] KEY_CONSIDERED B315D939A057EE4FFC869960C2242126A00AF865 0
[GNUPG:] KEY_CONSIDERED B315D939A057EE4FFC869960C2242126A00AF865 2
gpg: checking the trustdb
[GNUPG:] KEY_CONSIDERED A8F8B4BFA29016179D1DFA4ABB44DA091B258C40 0
[GNUPG:] KEY_CONSIDERED 41A0F38BFCCCD71A583B1019E47E06C9175C3412 0
[GNUPG:] KEY_CONSIDERED 37850F9C09C17E2BB1839CFA0035A47D0C2AD7E8 0
[GNUPG:] KEY_CONSIDERED 0C0524FF1FC0B68B0DB8EE4BAF95BFEFA2E76D41 0
[GNUPG:] KEY_CONSIDERED 2CEB551136BE1ED8EFD004214D0B3DD80B86DB54 0
[GNUPG:] KEY_CONSIDERED 7CF9AFC0F1F66888E5ACC586110497EFDE7F924D 0
[GNUPG:] KEY_CONSIDERED B315D939A057EE4FFC869960C2242126A00AF865 0
[GNUPG:] KEY_CONSIDERED 53F5923F6D54ECDA3EEBDF9E7593903C7525C468 0
[GNUPG:] KEY

'C:\\Users\\prashant\\Desktop\\PGP\\FTSol\\media\\encrypted\\terms.gpg'